# Python中的错误与异常

通常来说，程序中的错误至少包括两种：一种是语法错误，另一种是异常。

- 语法错误，是由于代码书写不规范导致程序无法将其正确的识别与运行。

- 异常，则是指程序的语法正确，可以被执行，但在执行的过程中遇到了错误，抛出异常。异常类型文档:[https://docs.python.org/3/library/exceptions.html#bltin-exceptions]


In [1]:
# 语法错误
if name is not None
    print(name)


SyntaxError: expected ':' (1236032539.py, line 1)

In [2]:
# 异常

10 / 0

ZeroDivisionError: division by zero

## 处理异常

```python
try:
    block
except error:
    block
else:
    block
```

程序监控try下面的执行语句，如果存在错误。就可以通过except进行捕获，如果except给定了某一特殊的错误名称，则只会捕获这一错误类型，发生别的错误时，程序还是会被异常终止。

也可以在except中放入多个错误类型进行捕获，except block 中只要有一个 exception 类型与实际匹配即可
```python
try:
    block
except (ValueError, IndexError) as err:
    block
```

```python
try:
    block
except ValueError as err:
    block
except IndexError as err:
    block
```

不过，很多时候，我们很难保证程序覆盖所有的异常类型，所以，更通常的做法，是在最后一个 except block，声明其处理的异常类型是 Exception。Exception 是其他所有非系统异常的基类，能够匹配任意非系统异常。也可以在 except 后面省略异常类型，这表示与任意异常相匹配（包括系统异常等）。
```python
try:
    block
except ValueError as err:
    block
except IndexError as err:
    block
except Exception as err:
    block

```

异常处理中，还有一个很常见的用法是 `finally`，经常和 `try`、`except` 放在一起来用。**无论发生什么情况，finally block 中的语句都会被执行，哪怕前面的 try 和 excep block 中使用了 return 语句**。

```python
import sys
try:
    f = open('file.txt', 'r')
    .... # some data processing
except OSError as err:
    print('OS error: {}'.format(err))
except:
    print('Unexpected error:', sys.exc_info()[0])
finally:
    f.close()

```
这段代码中，try block 尝试读取 file.txt 这个文件，并对其中的数据进行一系列的处理，到最后，无论是读取成功还是读取失败，程序都会执行 finally 中的语句——关闭这个文件流，确保文件的完整性。因此，在 finally 中，我们通常会放一些无论如何都要执行的语句。

## 用户自定义异常
实际工作中，如果内置的异常类型无法满足我们的需求，或者为了让异常更加详细、可读，想增加一些异常类型的其他功能，我们可以自定义所需异常类型。例如：
```python
class MyInputError(Exception):
    """Exception raised when there're errors in input"""
    def __init__(self, value): # 自定义异常类型的初始化
        self.value = value
    def __str__(self): # 自定义异常类型的string表达形式
        return ("{} is invalid input".format(repr(self.value)))
    
try:
    raise MyInputError(1) # 抛出MyInputError这个异常
except MyInputError as err:
    print('error: {}'.format(err))

```


## 异常的使用场景与注意点

通常来说，在程序中，如果我们不确定某段代码能否成功执行，往往这个地方就需要使用异常处理。

不过，有一点切记，我们不能走向另一个极端——滥用异常处理。

```python
d = {'name': 'jason', 'age': 20}
try:
    value = d['dob']
    ...
except KeyError as err:
    print('KeyError: {}'.format(err))

```

正常的 flow-control 逻辑，不要使用异常处理，直接用条件语句解决就可以了。上面的例子，可以直接改写为
```python
if 'dob' in d:
    value = d['dob']
    ...

```


# 思考
在异常处理时，如果 try block 中有多处抛出异常，需要我们使用多个 try except block 吗？以数据库的连接、读取为例，下面两种写法，你觉得哪种更好呢？

```python
try:
    db = DB.connect('<db path>') # 可能会抛出异常
    raw_data = DB.queryData('<viewer_id>') # 可能会抛出异常
except (DBConnectionError, DBQueryDataError) err:
    print('Error: {}'.format(err))

```

```python
try:
    db = DB.connect('<db path>') # 可能会抛出异常
    try:
        raw_data = DB.queryData('<viewer_id>')
    except DBQueryDataError as err:
         print('DB query data error: {}'.format(err))
except DBConnectionError as err:
     print('DB connection error: {}'.format(err))

```

## `raise`